### # Salvando o Data Frame em diferentes formatos

In [ ]:
# Verificar quais pastas existem dentro do diretório
display(dbutils.fs.ls('/Volumes/workspace/default/arquivos-aula'))

# Criar uma nova pasta
dbutils.fs.mkdirs('/Volumes/workspace/default/arquivos-aula/DiferentesFormatos')

### # Termos técnicos para manipulação de pastas e arquivos

# WRITE = Escrever/Gravar
# OVERWRITE = Sobrescreve/Substitui arquivo que já existe
# APPEND = Mantém o que já existe e adciona o novo conteúdo no final
# IGNORE = Salva somente se o local de destino não existir

### # Selecionar o SCHEMA (db_bikes), fazer uma consulta na tabela para depois criar o Data Frame e então salvá-lo em outros formatos como por exemplo PARQUET.

In [ ]:
USE db_bikes;
-- Consulta de contratos de clientes.

SELECT customer_id as Id,
first_name as Nome,
phone as Telefone,
email as Email
FROM clientes WHERE first_name like "G%"; -- Filtra os nomes que começam com a letra G.

### # Utilizando o Spark para ler Query SQL (apenas ler, ainda não salva em um Data Frame):

In [ ]:
spark.sql("""

SELECT
customer_id as Id,
first_name as Nome,
phone as Telefone,
email as Email
FROM clientes
WHERE first_name like "G%" -- Filtra os nomes que começam com a letra G.

"""
).show()

### # Salvando a Query em um Data Frame

In [ ]:
dfSQL = spark.sql("""     -- Dá um nome para o Data Frame (neste exemplo é dfSQL), e remove o .show() que servia para visualizar os dados selecionados.

SELECT
customer_id as Id,
first_name as Nome,
phone as Telefone,
email as Email
FROM clientes
WHERE first_name like "G%"

"""
)

In [ ]:
### # Visualizando o DF criado

display(dfSQL)

### # Salvando o Data Frame em formato PARQUET

dfSQL.write.mode("overwrite").parquet("/Volumes/workspace/default/arquivos-aula/DiferentesFormatos/formatoParquet") # Escreve o Data Frame no formato Parquet em que WRITE é o modo de escrita, e OVERWRITE é o modo de sobrescrita.

### # Salvando Data Frame em formato CSV e JSON

dfSQL.write.option("delimiter", ",").mode("overwrite").csv("/Volumes/workspace/default/arquivos-aula/DiferentesFormatos/formatoCSV") # Formato CSV
dfSQL.write.mode("overwrite").json("/Volumes/workspace/default/arquivos-aula/DiferentesFormatos/formatoJSON") # Formato JSON

display(dbutils.fs.ls('/Volumes/workspace/default/arquivos-aula/DiferentesFormatos')) # Verifica se os arquivos foram criados

In [ ]:
### # Organizar o código "quebrando em partes" com \

# Modo de escrita com \
    
dfSQL.write\
    .option("delimiter", ",")\
    .mode("overwrite")\
    .csv("/Volumes/workspace/default/arquivos-aula/DiferentesFormatos/formatoCSV")

# O resultado será o mesmo da forma de criar o Data Frame acima, porém é uma forma mais elegante de escrever o código e mais fácil de alterar alguma opção ou caminho caso seja necessário.

### # Ler o arquivo CSV

In [ ]:
# Caminho para o arquivo CSV
caminho_csv = "/Volumes/workspace/default/arquivos-aula/DiferentesFormatos/formatoCSV"

# Ler o arquivo CSV
df_csv = spark.read.csv(caminho_csv, header=False, inferSchema=True)

# Mostrar o conteúdo do DataFrame
display(df_csv)

### # Alterar (renomear) as colunas da tabela

df_csv = df_csv.withColumnRenamed("_c0", "Id")\
               .withColumnRenamed("_c1", "Nome")\
               .withColumnRenamed("_c2", "Telefone")\
               .withColumnRenamed("_c3", "E-mail")

display(df_csv)

### # Ler o arquivo JSON

In [ ]:
# Caminho para o arquivo JSON
caminho_json = "/Volumes/workspace/default/arquivos-aula/DiferentesFormatos/formatoJSON"

# Ler o arquivo JSON salvo no Data Frame
df_json = spark.read.json(caminho_json)

# Mostrar o conteúdo do Data Frame)
display(df_json)

### # Ler arquivo no formato PARQUET

In [ ]:
# Caminho para o arquivo PARQUET
caminho_parquet = "/Volumes/workspace/default/arquivos-aula/DiferentesFormatos/formatoParquet"

# Ler o arquivo PARQUET salvo no Data Frame
df_parquet = spark.read.parquet(caminho_parquet)

# Mostrar o conteúdo do Data Frame)
display(df_parquet)

### # Cria um novo DF para salvar dados particionados (apenas partes de uma tabela)

In [ ]:
# Cria um novo Data Frame da tabela clientes

novoDf = spark.sql('select * from db_bikes.clientes')
display(novoDf)

### # Ler o novo PARQUET particionado em forma de tabela (no exemplo abaixo somente os dados do estado de NY)

In [ ]:
# Caminho para o arquivo PARQUET
novo_parquet = "dbfs:/Volumes/workspace/default/arquivos-aula/DiferentesFormatos/particionado_formatoParquet/state=NY/" # Mostra somente do estado de NY

# Se alterar para CA ou TX ao invés de NY mostrará somente os que são dos estados selecionados.
# Se remover o /state=NY/ mostrará todos os arquivos do Data Frame normalmente.

# Ler o arquivo PARQUET salvo no Data Frame
df_Ny = spark.read.parquet(novo_parquet)

# Mostrar o conteúdo do Data Frame)
display(df_Ny)

### # Altera o nome das colunas da tabela

In [ ]:
df_Ny = df_Ny.withColumnRenamed("customer_id", "Id")\
             .withColumnRenamed("first_name", "Nome")\
             .withColumnRenamed("last_name", "Sobrenome")\
             .withColumnRenamed("phone", "Telefone")\
             .withColumnRenamed("email", "Email")\
             .withColumnRenamed("street", "Rua")\
             .withColumnRenamed("city", "Cidade")\
             .withColumnRenamed("zip_code", "CEP")

             # "nomeDaColunaVELHA", "nomeDaColunaNOVA"

display(df_Ny)